# 数据处理
转换为csv文件

In [16]:
import pandas as pd
import zl_classify
import imp
imp.reload(zl_classify)
from zl_classify import load_data_csvs,MLP_softmax,MLP_sigmoid
from sklearn.model_selection import cross_val_score
import sklearn as sk
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras import layers
from ufy.HX.diagnosis.classify import joblib
import keras
from ufy.HX.diagnosis.classify import KNN
from ufy.HX.diagnosis.classify import plot_roc_curve
import os
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score,confusion_matrix
from ufy.HX.diagnosis.classify import RandomForest
from ufy.HX.diagnosis.classify import plot_roc_curve
from ufy.HX.diagnosis.classify import XGboost
from zl_classify import MLP_softmax
from ufy.HX.diagnosis.classify import min_max_normal


In [17]:
name_data = 'train_data_name'
test_name = 'test_data_name'
data = pd.read_excel('train_data.xlsx',sheet_name='train_data')
test_data = pd.read_excel('test_data.xlsx',sheet_name='test_data')
data.to_csv(name_data+'.csv',index=False)
test_data.to_csv(test_name+'.csv',index=False)
labels = [0,1,2]

In [18]:
# 加载并划分数据集
filenames = [name_data+'.csv']
(x_train,y_train),(x_test,y_test) = load_data_csvs(filenames,
                                                  target_cols=1,# 最后一列是标签
                                                  drop_id=True,
                                                  id_cols=1,# 前面三列是无关特征
                                                  split_rate=0.6,
                                                  shuffle=True)
print((x_train.shape,y_train.shape),(x_test.shape,y_test.shape))
input_shape = x_train.shape[1]
def metrics_model(y_true,y_pre_cls,model_name,labels):
    acc = accuracy_score(y_true,y_pre_cls)
    prec = precision_score(y_true,y_pre_cls,average=None)
    f1 = f1_score(y_true,y_pre_cls,average=None)
    rec = recall_score(y_true,y_pre_cls,average=None)
    cof_met = confusion_matrix(y_true,y_pre_cls,labels=labels)
    result = {
        '模型':model_name,
        '准确率':acc,
        '精确率':prec,
        'F1':f1,
        '召回率':rec,
        '混淆矩阵':cof_met
    }
    return result

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.86it/s]

load data: TYY4data_22-03-23.csv
((1679, 478), (1679, 1)) ((187, 478), (187, 1))


In [66]:
x_train_normal = min_max_normal(x_train)
x_test_normal = min_max_normal(x_test)
y_true = y_test[:, 0]
model_path = 'model_weights/'+name_data+'/'
if os.path.exists(model_path) == False:
    os.makedirs(model_path)
total_result = pd.DataFrame(columns=['模型','准确率','精确率','F1','召回率','混淆矩阵'])

# KNN
knn = sk.neighbors.KNeighborsClassifier()
y_score = knn.fit(x_train,y_train)
y_pre_cls = knn.predict(x_test)
joblib.dump(knn, model_path+'knn.pkl')
result = metrics_model(y_true,y_pre_cls,'knn',labels)
total_result = total_result.append(result,ignore_index=True)

# Random Forest
model_path = 'model_weights/'+name_data+'/RandomForest.pkl'
rdfcls = RandomForest(x_train,y_train,model_savepath=model_path,istraining=True)
y_pre = rdfcls.predict_proba(x_test)[:,1]
y_pre_cls = rdfcls.predict(x_test)
result = metrics_model(y_true,y_pre_cls,'randomforest',labels)
total_result = total_result.append(result,ignore_index=True)

# XGBoost
model_path = 'model_weights/'+name_data+'/xgboost.pkl'
xgbc = XGboost(x_train,y_train,model_savepath=model_path,istraining=True)
y_pre = xgbc.predict_proba(x_test)
# print('y_pre:',y_pre)
y_pre_cls = xgbc.predict(x_test)
# print('y_cls:',y_pre_cls)

result = metrics_model(y_true,y_pre_cls,'xgboost',labels)
total_result = total_result.append(result,ignore_index=True)


# MLP_softmax
model_path = 'model_weights/'+name_data+'/MLP_softmax.h5'
mlp_softmax = MLP_softmax(x_train,y_train,model_savepath=model_path,istraining=True,batch_size=64,validation_split=0.2,class_num=len(labels),shape_in=input_shape)
y_pre_cls = mlp_softmax.predict(x_test)
y_pre_cls = np.argmax(y_pre_cls,axis=-1)
result = metrics_model(y_true,y_pre_cls,'MLP',labels)
total_result = total_result.append(result,ignore_index=True)
total_result.to_excel('result_'+name_data+'.xlsx')


C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Anaconda\lib\site-packages\ufy\HX\diagnosis\classify.py:226: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Administrator\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Anaconda\lib\sit

[10:40:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
(261, 1)
(261, 1, 3)
(261, 3)
Epoch 1/10000
4/4 [==============================] - 0s 40ms/step - loss: 0.0485 - acc: 0.9952 - val_loss: 1.9171 - val_acc: 0.8113
Epoch 2/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0688 - acc: 0.9808 - val_loss: 2.7301 - val_acc: 0.8302
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 7.4446e-07 - acc: 1.0000 - val_loss: 3.7491 - val_acc: 0.8302
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0041 - acc: 1.0000 - val_loss: 3.6913 - val_acc: 0.8302
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 0.0137 - acc: 0.9952 - val_loss: 2.8076 - val_acc: 0.830

In [19]:
# 加载并划分数据集
filenames = [test_name+'.csv']
undepend_test_data = pd.read_csv(filenames[0])
(_,_),(x_test,y_test) = load_data_csvs(filenames,
                                                  target_cols=1,# 最后一列是标签
                                                  drop_id=True,
                                                  id_cols=1,# 前面一列是无关特征
                                                  split_rate=0,
                                                  shuffle=False)
print((x_train.shape,y_train.shape),(x_test.shape,y_test.shape))
input_shape = x_train.shape[1]


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

load data: TYY4test_22-03-23.csv


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.25it/s]

((1679, 478), (1679, 1)) ((13986, 478), (13986, 1))


In [20]:
# x_train_normal = min_max_normal(x_train)
x_test_normal = min_max_normal(x_test)
y_true = y_test[:, 0]
model_path = 'model_weights/'+name_data+'/'
if os.path.exists(model_path) == False:
    os.makedirs(model_path)
total_result = pd.DataFrame(columns=['模型','准确率','精确率','F1','召回率','混淆矩阵'])

# KNN
knn = sk.neighbors.KNeighborsClassifier()
y_score = knn.fit(x_train,y_train)
y_pre_cls = knn.predict(x_test)
joblib.dump(knn, model_path+'knn.pkl')
result = metrics_model(y_true,y_pre_cls,'knn',labels)
total_result = total_result.append(result,ignore_index=True)

# Random Forest
model_path = 'model_weights/'+name_data+'/RandomForest.pkl'
rdfcls = RandomForest(x_train,y_train,model_savepath=model_path,istraining=True)
y_pre = rdfcls.predict_proba(x_test)[:,1]
y_pre_cls = rdfcls.predict(x_test)
result = metrics_model(y_true,y_pre_cls,'randomforest',labels)
total_result = total_result.append(result,ignore_index=True)

# XGBoost
model_path = 'model_weights/'+name_data+'/xgboost.pkl'
xgbc = XGboost(x_train,y_train,model_savepath=model_path,istraining=True)
# y_pre = xgbc.predict_proba(x_test)[:,1]
y_pre = xgbc.predict_proba(x_test)
undepend_test_data['xgboost_pred_proba0'] = y_pre[:,0]
undepend_test_data['xgboost_pred_proba1'] = y_pre[:,1]
undepend_test_data['xgboost_pred_proba2'] = y_pre[:,2]
undepend_test_data['xgboost_pred_proba3'] = y_pre[:,3]
y_pre_cls = xgbc.predict(x_test)
undepend_test_data['xgboost_pred'] = y_pre_cls
result = metrics_model(y_true,y_pre_cls,'xgboost',labels)
total_result = total_result.append(result,ignore_index=True)


# MLP_softmax
model_path = 'model_weights/'+name_data+'/MLP_softmax.h5'
mlp_softmax = MLP_softmax(x_train,y_train,model_savepath=model_path,istraining=True,batch_size=64,validation_split=0.2,class_num=len(labels),shape_in=input_shape)
y_pre_cls = mlp_softmax.predict(x_test)
y_pre_cls = np.argmax(y_pre_cls,axis=-1)
result = metrics_model(y_true,y_pre_cls,'MLP',labels)
total_result = total_result.append(result,ignore_index=True)
total_result.to_excel('result_'+test_name+'.xlsx')
undepend_test_data.to_excel('xgboost_result_'+test_name+'.xlsx',index=False)

C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Anaconda\lib\site-packages\ufy\HX\diagnosis\classify.py:226: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
C:\Users\Administrator\AppData\Roaming\Python\Python38\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Anaconda\lib\sit

[16:43:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
(1679, 1)
(1679, 1, 4)
(1679, 4)
Epoch 1/10000
21/21 [==============================] - 0s 8ms/step - loss: 0.1451 - acc: 0.9531 - val_loss: 0.1214 - val_acc: 0.9732
Epoch 2/10000
21/21 [==============================] - 0s 4ms/step - loss: 0.0682 - acc: 0.9792 - val_loss: 0.0762 - val_acc: 0.9792
Epoch 3/10000
21/21 [==============================] - 0s 4ms/step - loss: 0.0481 - acc: 0.9836 - val_loss: 0.0934 - val_acc: 0.9732
Epoch 4/10000
21/21 [==============================] - 0s 4ms/step - loss: 0.0433 - acc: 0.9829 - val_loss: 0.0981 - val_acc: 0.9702
Epoch 5/10000
21/21 [==============================] - 0s 5ms/step - loss: 0.0543 - acc: 0.9784 - val_loss: 0.1285 - val_acc

In [9]:
y_pre

array([[0.28311256, 0.08236288, 0.6345245 ],
       [0.68077374, 0.28495133, 0.0342749 ],
       [0.7063562 , 0.08121357, 0.21243024],
       ...,
       [0.18981019, 0.6475031 , 0.16268672],
       [0.06001392, 0.6394642 , 0.30052185],
       [0.0676529 , 0.654036  , 0.27831113]], dtype=float32)